# Install Packages

In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Restart the kernel by running the below code snippet or clicking the refresh button restart kernel at the top, followed by clicking Restart button.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

Run the following code snippet in the next cell to import the libraries.

In [2]:
import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

# Simple Function Calling

### Why function calling?
When working with a generative text model, it can be difficult to coerce the LLM to give consistent responses in a structured format such as JSON. Function calling makes it easy to work with LLMs via prompts and unstructured inputs, and have the LLM return a structured response that can be used to call an external function.

You can think of function calling as a way to get structured output from user prompts and function definitions, use that structured output to make an API request to an external system, then return the function response to the LLM to generate a response to the user. In other words, function calling in Gemini extracts structured parameters from unstructured text or messages from users.

### Use the Gemini Pro model
The Gemini Pro (gemini-pro) model is designed to handle natural language tasks, multiturn text and code chat, and code generation.

Load the Gemini Pro model:

In [7]:
model = GenerativeModel("gemini-2.0-flash-001")

Simple function calling
We'll use function calling to set up a weather API request for users to obtain the current conditions in a given location. Function parameters are specified as a Python dictionary in accordance with the [OpenAPI JSON schema format](https://spec.openapis.org/oas/v3.0.3#schemawr).

Run the following code snippet to specify function declaration and parameters needed to make a request for our weather API.

In [8]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)

Run the following code snippet to define a tool for the LLM to call that includes the `get_current_weather_func`.

In [9]:
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

We will then instruct the model to generate content. Include the `tool` that you just created to generate a response:

In [10]:
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  avg_logprobs: -2.4811893776391765e-06
}
usage_metadata {
  prompt_token_count: 25
  candidates_token_count: 7
  total_token_count: 32
  prompt_tokens_details {
    modality: TEXT
    token_count: 25
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 7
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1749014306
  nanos: 370469000
}
response_id: "Itc_aKXOFv2fmecPlYPsgQI"

Let's inspect the function call portion of the response

In [11]:
response.candidates[0].content.parts[0].function_call

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Boston"
    }
  }
}

The generated response includes a function signature that can be used to call the weather API. Now, we have everything that we need to form a request body and make an API call to an external system.

# Complex Function Calling

Let's generate a function call that has a more complex structure. Let's use the function response to make an API call that converts an address to latitude and longitude coordinates.

Run the following code snippet to define a function declaration within a tool:

In [12]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Get latitude and longitude for a given location",
    parameters={
    "type": "object",
    "properties": {
        "poi": {
            "type": "string",
            "description": "Point of interest"
        },
        "street": {
            "type": "string",
            "description": "Street name"
        },
        "city": {
            "type": "string",
            "description": "City name"
        },
        "county": {
            "type": "string",
            "description": "County name"
        },
        "state": {
            "type": "string",
            "description": "State name"
        },
        "country": {
            "type": "string",
            "description": "Country name"
        },
        "postal_code": {
            "type": "string",
            "description": "Postal code"
        },
    },
},
)
location_tool = Tool(
    function_declarations=[get_location],
)

Next, call the model to generate a response.

In [13]:
prompt = """
I want to get the lat/lon coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""
response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[location_tool],
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_location"
  args {
    fields {
      key: "city"
      value {
        string_value: "Mountain View"
      }
    }
    fields {
      key: "country"
      value {
        string_value: "US"
      }
    }
    fields {
      key: "postal_code"
      value {
        string_value: "94043"
      }
    }
    fields {
      key: "state"
      value {
        string_value: "CA"
      }
    }
    fields {
      key: "street"
      value {
        string_value: "1600 Amphitheatre Pkwy"
      }
    }
  }
}

Then, extract the results from the function response and make an API call.

In [21]:
import json

In [23]:
x = response.candidates[0].content.parts[0].function_call.args
# print(x)
url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (kHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
x = requests.get(url, headers=headers)
# print(x.status_code)
# print(x.text)
content = x.json()
content

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Great work! You were able to construct a function and tool that the LLM used to output the parameters necessary for a function call, then you actually made the function call to obtain the coordinates of the specified location.

Here we used the [OpenStreetMap Nominatim API](https://nominatim.openstreetmap.org/ui/search.html) to geocode an address to make it easy to use and learn in this notebook. If you're working with large amounts of maps or geolocation data, you can use the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding).

# Function calling in a chat session

Next, let's use the chat model in Gemini to help customers get information about products in a store.

Let's start by defining multiple functions within a tool for getting the product information, the location of stores and placing an order.

In [24]:
get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Get the SKU for a product",
    parameters={
    "type": "object",
    "properties": {
        "product_name": {
            "type": "string",
            "description": "Product name"
        }
    }
},
)
get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)
place_order_func = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
    "type": "object",
    "properties": {
        "product": {
            "type": "string",
            "description": "Product name"
        },
        "account": {
            "type": "integer",
            "description": "Account number"
        },
        "address": {
            "type": "string",
            "description": "Shipping address"
        }
    }
},
)
retail_tool = Tool(
    function_declarations=[get_product_info_func, 
                           get_store_location_func, 
                           place_order_func,
                          ],
)

Function calling can also be used in a multi-turn chat session. You can specify tools when creating a model to avoid having to send them with every request.

In [26]:
model = GenerativeModel("gemini-2.0-flash-001", 
                        generation_config={"temperature": 0},
                        tools=[retail_tool])
chat = model.start_chat()

Start the conversation by asking if they have a certain product in stock.

In [27]:
prompt = """
Do you have the Pixel 8 Pro in stock?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_sku"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

As expected, the response includes a structured function call that we can use to communicate with external systems.

In reality, you would execute function calls against an external system or database. Since this lab focuses on the ability to extract function parameters and generate function calls, you'll use mock data to feed responses back to the model rather than using an actual API server.

Use synthetic data to simulate a response payload from an external API.

In [28]:
api_response = {"sku": "GA04834-US", "in_stock": "yes"}

Let's include details from the external API call and generate a response to the user.

In [29]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_sku",
        response={
            "content": api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, we have the Pixel 8 Pro in stock. The SKU is GA04834-US.\n"

The user might ask where they can buy a phone from a nearby store.

In [30]:
prompt = """
Where can I buy one near Mountain View, CA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

We get a response with another structured function call, this time it's set up to use a different function from our tool.

Use synthetic data to simulate a response payload from an external API.

In [31]:
api_response = {"store": "1600 Amphitheatre Pkwy, Mountain View, CA 94043, US"}

Again, let's include details from the external API call and generate a response to the user.

In [32]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "The closest store is located at 1600 Amphitheatre Pkwy, Mountain View, CA 94043, US.\n"

Finally, the user might ask to order a phone and have it shipped to an address.

In [33]:
prompt = """
I’d like to place an order for a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089, using account number 101.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "account"
      value {
        number_value: 101.0
      }
    }
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

Perfect! We extracted the user's desired product, address, fetched their account number, and now we can call an API to place the order:

Use synthetic data to simulate a response payload from an external API.

In [34]:
api_response = {"payment_status": "paid", "order_number": 12345, "est_arrival": "2 days"}

Let's include details from the external API call and generate a response to the user.

In [35]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "OK. I have placed an order for a Pixel 8 Pro to be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089, using account number 101. The order number is 12345, the payment status is paid, and the estimated arrival is in 2 days.\n"